## TP1

# Sous-tache 1a. Pre-traitement des questions

importation des librairies

In [38]:
%pip install --upgrade certifi
%pip install --upgrade jellyfish
%pip install --upgrade colorama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
from __future__ import division  # Python 2 users only
import nltk, re, pprint
from nltk import word_tokenize
import spacy
import ssl
from spacy import displacy
from SPARQLWrapper import SPARQLWrapper, JSON



In [42]:
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lemoi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lemoi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [43]:
!python -m spacy download en_core_web_sm
NER = spacy.load("en_core_web_sm")

     --------------------------------------- 12.8/12.8 MB 50.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-11-28 11:16:22.670577: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-11-28 11:16:22.670896: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-28 11:16:29.127281: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-11-28 11:16:29.128386: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-11-28 11:16:29.129349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-11-28 11:16:29.130200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

In [44]:
doc = open('questions.xml').read()
phrase =re.search('<string lang="en">(.*?)</string>', doc).group(1)
print(phrase)

Which river does the Brooklyn Bridge cross?


In [45]:
tokens = word_tokenize(phrase)
print(tokens)

['Which', 'river', 'does', 'the', 'Brooklyn', 'Bridge', 'cross', '?']


In [46]:
nltk.pos_tag(tokens)

[('Which', 'JJ'),
 ('river', 'NN'),
 ('does', 'VBZ'),
 ('the', 'DT'),
 ('Brooklyn', 'NNP'),
 ('Bridge', 'NNP'),
 ('cross', 'NN'),
 ('?', '.')]

# Sous-tache 1b. Identifier le type de réponse attendu

In [47]:
# Récupération des données dans le document question.xml
doc = open('questions.xml').read()
#Récupération de toute les phrases comprises entre les balises "<string lang="en">" et "</string>"
questions = re.findall('<string lang="en">(.*?)</string>', doc)

newList = []
for question in questions :
    # Remplacement des strings <![CDATA[ et ]]> en string vide
    if '<![CDATA[' in question:
        question = question.replace("<![CDATA[", "")
        question = question.replace("]]>", "")
    #Ajout de chaque phrase dans le tableau
    newList.append(question)
# récupération de toute les phrases comprises entre les balises "<query>" et "</query>"
questionQuery = re.findall('<query>(.*?)</query>',doc, re.DOTALL)
questionsQuery = []
# Remplacement de tout les strings ci-dessous par un expace vide
for value in questionQuery:
    if 'uri\nWHERE' in value:
        value = value.replace("uri\nWHERE","uri WHERE")
    if '?uri WHERE' in value:
        value = value.replace("uriWHERE","uri WHERE")
    if '\n' in value : 
        value = value.replace("\n", "")
    if '<![CDATA[' in value:
        value = value.replace("<![CDATA[", "")
        value = value.replace("]]>", "")
    #Ajout de chaque phrase dans le tableau
    questionsQuery.append(value)

# Affichage des questions
print(newList)
print("\n\n-------\n\n")
# Affichage des requetes query du document question
print(questionsQuery)

['Which river does the Brooklyn Bridge cross?', 'Who is the author of Wikipedia?', 'In which country does the Nile start?', 'What is the highest place of Karakoram?', 'Who designed the Brooklyn Bridge?', 'Who created Goofy?', 'Who is the mayor of New York City?', 'Which is the source of the Yenisey river', 'Which museum exhibits The Scream by Munch?', 'Which states border Illinois?', 'Who was the wife of U.S. president Lincoln?', 'In which programming language is GIMP written?', 'In which country is the Limerick Lake?', 'What is the currency of the Czech Republic?', 'Who developed the video game World of Warcraft?', 'Who founded Aldi?', 'How many employees does IBM have?', 'What is the area code of Berlin?', 'When was the Battle of Gettysburg?', 'What is the official languages of Italy?', 'Who wrote the book The Pillars of the Earth?', 'Who is the author of the WikiLeaks association?', 'Give me all actors starring in Last Action Hero.', 'Who is the owner of Universal Studios Lot?', 'Wh

In [48]:
# Séparation de tout les mots de chaque phrase 
tokens = [word_tokenize(x) for x in newList]
print(*tokens)

['Which', 'river', 'does', 'the', 'Brooklyn', 'Bridge', 'cross', '?'] ['Who', 'is', 'the', 'author', 'of', 'Wikipedia', '?'] ['In', 'which', 'country', 'does', 'the', 'Nile', 'start', '?'] ['What', 'is', 'the', 'highest', 'place', 'of', 'Karakoram', '?'] ['Who', 'designed', 'the', 'Brooklyn', 'Bridge', '?'] ['Who', 'created', 'Goofy', '?'] ['Who', 'is', 'the', 'mayor', 'of', 'New', 'York', 'City', '?'] ['Which', 'is', 'the', 'source', 'of', 'the', 'Yenisey', 'river'] ['Which', 'museum', 'exhibits', 'The', 'Scream', 'by', 'Munch', '?'] ['Which', 'states', 'border', 'Illinois', '?'] ['Who', 'was', 'the', 'wife', 'of', 'U.S.', 'president', 'Lincoln', '?'] ['In', 'which', 'programming', 'language', 'is', 'GIMP', 'written', '?'] ['In', 'which', 'country', 'is', 'the', 'Limerick', 'Lake', '?'] ['What', 'is', 'the', 'currency', 'of', 'the', 'Czech', 'Republic', '?'] ['Who', 'developed', 'the', 'video', 'game', 'World', 'of', 'Warcraft', '?'] ['Who', 'founded', 'Aldi', '?'] ['How', 'many', 'em

In [49]:
tag= []
# Ajout des tag de type de mot apres chaque mot pour les identifier
for token in tokens :
    print(nltk.pos_tag(token))

[('Which', 'JJ'), ('river', 'NN'), ('does', 'VBZ'), ('the', 'DT'), ('Brooklyn', 'NNP'), ('Bridge', 'NNP'), ('cross', 'NN'), ('?', '.')]
[('Who', 'WP'), ('is', 'VBZ'), ('the', 'DT'), ('author', 'NN'), ('of', 'IN'), ('Wikipedia', 'NNP'), ('?', '.')]
[('In', 'IN'), ('which', 'WDT'), ('country', 'NN'), ('does', 'VBZ'), ('the', 'DT'), ('Nile', 'NNP'), ('start', 'NN'), ('?', '.')]
[('What', 'WP'), ('is', 'VBZ'), ('the', 'DT'), ('highest', 'JJS'), ('place', 'NN'), ('of', 'IN'), ('Karakoram', 'NNP'), ('?', '.')]
[('Who', 'WP'), ('designed', 'VBD'), ('the', 'DT'), ('Brooklyn', 'NNP'), ('Bridge', 'NNP'), ('?', '.')]
[('Who', 'WP'), ('created', 'VBD'), ('Goofy', 'NNP'), ('?', '.')]
[('Who', 'WP'), ('is', 'VBZ'), ('the', 'DT'), ('mayor', 'NN'), ('of', 'IN'), ('New', 'NNP'), ('York', 'NNP'), ('City', 'NNP'), ('?', '.')]
[('Which', 'WDT'), ('is', 'VBZ'), ('the', 'DT'), ('source', 'NN'), ('of', 'IN'), ('the', 'DT'), ('Yenisey', 'NNP'), ('river', 'NN')]
[('Which', 'NNP'), ('museum', 'NN'), ('exhibits'

In [50]:
# la méthode identifier permet d'identifier quelle type de question est posé (where, what, how ...)
def identifier(str):
    if(re.search("When",str, re.IGNORECASE)):
        return "When"
    elif(re.search("Who",str, re.IGNORECASE)):
        return "Who"
    elif(re.search("Where",str, re.IGNORECASE)):
        return "Where"
    elif(re.search("What",str, re.IGNORECASE)):
        return "What"
    elif(re.search("How",str, re.IGNORECASE)):
        return "How"
    elif(re.search("In which",str, re.IGNORECASE)):
        return "Where"
    elif(re.search("Which",str, re.IGNORECASE)):
        return "What"


In [51]:
# npp_old est une méthode de récupération des entités de la base mentionné dans la question
def nnp_old(str):
    val = []
    tokens = nltk.word_tokenize(str)
    # J'utilise les tokens pour identifier les mots NNP et je récupere tout les mot qui se trouve entre 2 NPP (The Pillars of the Earth)
    for (word, tag) in nltk.pos_tag(tokens):
        if re.search("NNP", tag):
            val.append(word)

    lengthWord = len(val)
    if lengthWord == 1 :
        value = val[0]
    else:
        # Suppréssion des espaces par des _
        value = re.findall(val[0]+ '.*?' + val[lengthWord-1], str)    
        value = "_".join(value).replace(' ', '_') 
    
    return value


In [52]:
# nnp a le meme but que nnp_old mais apres plus test elle à des résulstats inférieur a nnp_old
# elle utilise néanmoins la librairie spacy qui repere tout seule les entité mais pas très fiable dans notre cas
def nnp(str):
    text = NER(str)
    for word in text.ents:
        return word.text

for list in newList:
    val = nnp(list)
    print(val)

Brooklyn Bridge
Wikipedia
Nile
Karakoram
the Brooklyn Bridge
Goofy
New York City
Yenisey
Munch
Illinois
U.S.
None
the Limerick Lake
the Czech Republic
World of Warcraft
Aldi
IBM
Berlin
Gettysburg
Italy
The Pillars of the Earth
WikiLeaks
None
Universal Studios Lot
Bruce Carver


In [53]:
#Cette méthode permet de récuperer toute les relations se trouvant dans le fichier relation.txt
# elle sépare dans un tableau à double dimension le type de relation et la relation
def extract_relations(path):
    f_read = open(f'{path}.txt', 'r').read()
    def get(pattern, array):
        res = re.findall(pattern, array)
        return [value.strip() for value in res]
    def typage(type, value):
        return (type, value.strip())
    relation_DBO = get('dbo:(.+?)\n', f_read)
    relation_DBP = get('dbp:(.+?)\n', f_read)
    res = [typage('dbo', value) for value in relation_DBO] + \
        [typage('dbp', value) for value in relation_DBP]
    return res

In [54]:
# initialisation de la variable relations avec toute les relations
relations = extract_relations("relations")
print(relations)

[('dbo', 'album'), ('dbo', 'areaCode'), ('dbo', 'battle'), ('dbo', 'birthDate'), ('dbo', 'birthPlace'), ('dbo', 'capital'), ('dbo', 'child'), ('dbo', 'city'), ('dbo', 'country'), ('dbo', 'creator'), ('dbo', 'crosses'), ('dbo', 'currency'), ('dbo', 'deathCause'), ('dbo', 'deathDate'), ('dbo', 'deathPlace'), ('dbo', 'developer'), ('dbo', 'director'), ('dbo', 'elevation'), ('dbo', 'formationYear'), ('dbo', 'foundedBy'), ('dbo', 'foundationPlace'), ('dbo', 'foundingYear'), ('dbo', 'genre'), ('dbo', 'governmentType'), ('dbo', 'ground'), ('dbo', 'height'), ('dbo', 'highest'), ('dbo', 'isPartOf'), ('dbo', 'keyPerson'), ('dbo', 'language'), ('dbo', 'largestCity'), ('dbo', 'league'), ('dbo', 'locatedInArea'), ('dbo', 'location'), ('dbo', 'numberOfEmployees'), ('dbo', 'officialLanguage'), ('dbo', 'operator'), ('dbo', 'orderInOffice'), ('dbo', 'owner'), ('dbo', 'producer'), ('dbo', 'programmingLanguage'), ('dbo', 'publisher'), ('dbo', 'seasonNumber'), ('dbo', 'series'), ('dbo', 'spokenIn'), ('dbo

In [55]:
# Récupération de tout les synonymes de chaque relation via le package nltk.corpus
from nltk.corpus import wordnet
d = {}

synonymes = []
for type, value in relations :
    if len(wordnet.synsets(value)) > 0 :
        synonymes.append(wordnet.synsets(value))
        d[(type,value)] = wordnet.synsets(value)




In [56]:
# méthode de calcul de la distance de chaque mot d'une phrase avec tout les synonyme des relations.
#la méthode retourne donc la relation la plus approprié par rapport a notre phrase. Mais dans notre cas il y a mieux et plus facile a mettre en place
def getDistance(question):
    maxSimilarity = 0
    synonymeSimilarity = ''
    wordSimilarity = ''
    
    for wordSentence in question:   
        if len(wordnet.synsets(wordSentence)) > 0 :
            for key, syn in d.items() :
                if len(syn) > 0 :
                    for s in syn :
                        #print(wordnet.synsets(wordSentence))
                        similarity = s.path_similarity(wordnet.synsets(wordSentence)[0])
                        if similarity > maxSimilarity :
                            maxSimilarity = similarity
                            synonymeSimilarity = s
                            wordSimilarity = wordSentence
                            relation = key
                else :
                    similarity = syn.path_similarity(wordnet.synsets(wordSentence))
                    if similarity > maxSimilarity :
                        maxSimilarity = similarity
                        synonymeSimilarity = syn
                        wordSimilarity = wordSentence
                        relation = key
                



    (type, relation) =  relation

    res = type+":"+relation
    return res





In [57]:
# dis et la méthode qui nous a permis de répondre au plus de question possible. mieux que nltk.distance qui était bloqué avec un string de meme taille
# jellyfish permet de comparer 2 string de taille différente. Dans notre cas je compare juste chaque mot des questions sur chaque mot des relations et
# je récupère la relations la plus haute 
import jellyfish
def dis(questions):
    bestVal =0
    val = 0
    key =''
    word = ''
    for wordSentence in questions: 
        for type,relation in relations:            
            val = jellyfish.jaro_distance(str(wordSentence),str(relation))
            if(val > bestVal):
                bestVal = val
                key = type
                word=relation
    res = key+':'+word
    return res
            


In [58]:
NER = spacy.load("en_core_web_sm")


In [75]:
listeResponseQuestionXML = []
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# Méthode qui permet de faire appel à l'api DBPedia et de récuperer les réponses.    
def getResponseQuestionXML(quest) :
    sparql.setQuery(quest)
    sparql.setReturnFormat(JSON)

    try:
        listResponseQuestionXML = []
        ret = sparql.queryAndConvert()
        print(r["uri"]["value"])
        for r in ret["results"]["bindings"]:
            if(len(r["uri"]["value"]) > 1) :
                listResponseQuestionXML.append(r["uri"]["value"])
    except Exception as e:
        print(e)
    return listResponseQuestionXML

In [85]:
listeResponseQuestionXML = []
 # méthode qui permet d'envoyer et de récuperer la réponse de chaque question sur l'api DBPedia   
def getResponseXML(request) :
    sparql.setQuery(request)

    sparql.setReturnFormat(JSON)
    results = sparql.query()
    #response = results.response.read()

    try:
        listResponseQuestion = []
        ret = sparql.queryAndConvert()
        for r in ret["results"]["bindings"]:
            if(len(r["uri"]["value"]) > 1) :
                listResponseQuestion.append(r["uri"]["value"])
    except Exception as e:
        print(e)
    return listResponseQuestion

In [86]:
# On passe en parametre chaque query du fichier question.xml est on stocke les résultats de chaque question
for val in questionsQuery:
    listeResponseQuestionXML.append(getResponseXML(val))

print(listeResponseQuestionXML)

'uri'
'uri'
'uri'
[['http://dbpedia.org/resource/East_River'], [], ['http://dbpedia.org/resource/Ethiopia', 'http://dbpedia.org/resource/Lake_Tana'], ['http://dbpedia.org/resource/K2'], ['http://dbpedia.org/resource/John_Augustus_Roebling'], ['http://dbpedia.org/resource/Bob_Ogle', 'http://dbpedia.org/resource/Paul_Murry'], ['Eric Adams'], ['http://dbpedia.org/resource/Ka-Hem', 'The most distant source: Yenisey-Angara-Selenga-Ider system'], ['National Gallery and Munch Museum'], ['http://dbpedia.org/resource/Kentucky', 'http://dbpedia.org/resource/Iowa', 'http://dbpedia.org/resource/Missouri', 'http://dbpedia.org/resource/Wisconsin', 'http://dbpedia.org/resource/Indiana'], ['http://dbpedia.org/resource/Mary_Todd_Lincoln'], ['http://dbpedia.org/resource/C_(programming_language)'], ['http://dbpedia.org/resource/Canada'], ['http://dbpedia.org/resource/Czech_koruna'], ['http://dbpedia.org/resource/Blizzard_Entertainment'], ['http://dbpedia.org/resource/Theo_Albrecht', 'http://dbpedia.org/r

In [87]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# Méthode de création pour nos query qui va permettre de répondre au question du fichier question.xml
def createQuery(question):
    # Mise en forme de la requete query
    query = """
    PREFIX XXXX
    PREFIX res: <http://dbpedia.org/resource/>
    SELECT DISTINCT ?uri WHERE {
        res:YYYY ZZZZ ?uri .
    }
    """
    # Récupération du mot le plus proche par rapport au relation
    res = dis(word_tokenize(question))
    # si le tableau contien o on remplace les xxxx de la query par une addresse qui fini par ontology
    if(res[2] == 'o'):
        query = query.replace("XXXX", "dbo: <http://dbpedia.org/ontology/>")
    # Sinon on récupere l'adresse property
    else :
        query = query.replace("XXXX", "dbp: <http://dbpedia.org/property/>")

    # on récupère les entités de chaque question qu'on replace insère dans la requetes
    text = nnp_old(question)
    query = query.replace("YYYY", text.replace(" ","_"))
    # remplacement des zzzz par la relation pour la question
    query = query.replace("ZZZZ", res)
    print(query)

    sparql.setQuery(query)
    return sparql

In [88]:
listResponseProject = []
 # méthode qui permet d'envoyer et de récuperer la réponse de chaque question sur l'api DBPedia   
def getResponseProject(request) :
    sparql = createQuery(request)

    sparql.setReturnFormat(JSON)
    results = sparql.query()
    #response = results.response.read()

    try:
        listResponseQuestion = []
        ret = sparql.queryAndConvert()
        for r in ret["results"]["bindings"]:
            if(len(r["uri"]["value"]) > 1) :
                listResponseQuestion.append(r["uri"]["value"])
    except Exception as e:
        print(e)
    return listResponseQuestion
    

In [89]:
# ajout de chaque réponse au question dans un tableau
for val in newList:
    listResponseProject.append(getResponseProject(val))


    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX res: <http://dbpedia.org/resource/>
    SELECT DISTINCT ?uri WHERE {
        res:Brooklyn_Bridge dbo:crosses ?uri .
    }
    

    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX res: <http://dbpedia.org/resource/>
    SELECT DISTINCT ?uri WHERE {
        res:Wikipedia dbp:authors ?uri .
    }
    

    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX res: <http://dbpedia.org/resource/>
    SELECT DISTINCT ?uri WHERE {
        res:Nile dbo:country ?uri .
    }
    

    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX res: <http://dbpedia.org/resource/>
    SELECT DISTINCT ?uri WHERE {
        res:Karakoram dbo:highest ?uri .
    }
    

    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX res: <http://dbpedia.org/resource/>
    SELECT DISTINCT ?uri WHERE {
        res:Brooklyn_Bridge dbp:designer ?uri .
    }
    

    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX res: <http://dbpedia.org/resource

In [95]:
# Comparaison des réponses entre la réponse des querys déjà préparé et la réponse a nos question

nbQuestionsCorrect = 0
nbQuestionsFalse = 0

for i in range(len(listeResponseQuestionXML)) :
    nbGoodAnswer = 0
    nbFalseAnswer = 0
    if listeResponseQuestionXML[i] == listResponseProject[i] :
        nbGoodAnswer += 1
        print('--------------------------------------------------------------------')
        print('Correct')
        print('Réponse des questions XML : ' + str(listeResponseQuestionXML[i]))
        print('Notre réponse : ' + str(listResponseProject[i]))
    else :
        nbFalseAnswer += 1
        print('--------------------------------------------------------------------')
        print('Fausse')
        print('Réponse des questions XML : ' + str(listeResponseQuestionXML[i]))
        print('Notre réponse : ' + str(listResponseProject[i]))
    print('\n')
    if nbFalseAnswer == 0 and nbGoodAnswer > 0:
        nbQuestionsCorrect += 1
    else : 
        nbQuestionsFalse +=1

print(f"notation : {nbQuestionsCorrect}/25")



--------------------------------------------------------------------
Correct
Réponse des questions XML : ['http://dbpedia.org/resource/East_River']
Notre réponse : ['http://dbpedia.org/resource/East_River']


--------------------------------------------------------------------
Fausse
Réponse des questions XML : []
Notre réponse : ['http://dbpedia.org/resource/Larry_Sanger', 'http://dbpedia.org/resource/Jimmy_Wales']


--------------------------------------------------------------------
Fausse
Réponse des questions XML : ['http://dbpedia.org/resource/Ethiopia', 'http://dbpedia.org/resource/Lake_Tana']
Notre réponse : []


--------------------------------------------------------------------
Correct
Réponse des questions XML : ['http://dbpedia.org/resource/K2']
Notre réponse : ['http://dbpedia.org/resource/K2']


--------------------------------------------------------------------
Correct
Réponse des questions XML : ['http://dbpedia.org/resource/John_Augustus_Roebling']
Notre réponse : ['